In [76]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from keras import backend as K
import cv2
import matplotlib.pyplot as plt
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.optimizers import SGD
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
train_dir="/kaggle/input/signature-verification-dataset/sign_data/train"
train_csv="/kaggle/input/signature-verification-dataset/sign_data/train_data.csv"
test_csv="/kaggle/input/signature-verification-dataset/sign_data/test_data.csv"
test_dir="/kaggle/input/signature-verification-dataset/sign_data/test"

In [4]:
df_train=pd.read_csv(train_csv)
df_train.sample(10)

,068/09_068.png,068_forg/03_0113068.PNG,1
16912,020/05_020.png,020/09_020.png,0
18187,015/015_18.PNG,015_forg/0213015_04.png,1
12678,049/01_049.png,049_forg/04_0206049.PNG,1
20464,056/06_056.png,056/02_056.png,0
1562,045/09_045.png,045/05_045.png,0
13788,018/03_018.png,018_forg/02_0106018.PNG,1
7408,065/02_065.png,065_forg/02_0206065.PNG,1
1826,038/02_038.png,038/06_038.png,0
19205,042/04_042.png,042/07_042.png,0
3921,014/014_03.PNG,014_forg/0214014_03.png,1


In [5]:
def dataset_train(train_csvfile):
    x1=[]
    x2=[]
    y_train=[]
    for i in range(0,2000):
        image1_path=os.path.join(train_dir,train_csvfile.iat[i,0])
        image2_path=os.path.join(train_dir,train_csvfile.iat[i,1])
        img1=cv2.imread(image1_path)
        img1= cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        img1=cv2.resize(img1,(150,150))
        x1.append(img1)
        img2=cv2.imread(image2_path)
        img2= cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
        img2=cv2.resize(img2,(150,150))
        x2.append(img2)
        y_train.append(train_csvfile.iat[i,2])
#         print(i)
#         l.append([img1,img2,train_csvfile.iat[i,2]])
    x1=np.array(x1).astype(np.float32)/255.0
    x2=np.array(x2).astype(np.float32)/255.0
    y_train=np.array(y_train).astype(np.float32)
    
    return x1,x2,y_train

In [6]:
def dataset_test(test_csvfile):
    x1=[]
    x2=[]
    y_train=[]
    for i in range(2000,4000):
        image1_path=os.path.join(test_dir,test_csvfile.iat[i,0])
        image2_path=os.path.join(test_dir,test_csvfile.iat[i,1])
        img1=cv2.imread(image1_path)
        img1= cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        img1=cv2.resize(img1,(150,150))
        x1.append(img1)
        img2=cv2.imread(image2_path)
        img2= cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
        img2=cv2.resize(img2,(150,150))
        x2.append(img2)
        y_train.append(test_csvfile.iat[i,2])
#         print(i)
#         l.append([img1,img2,train_csvfile.iat[i,2]])
    x1=np.array(x1).astype(np.float32)/255.0
    x2=np.array(x2).astype(np.float32)/255.0
    y_train=np.array(y_train).astype(np.float32)
    
    return x1,x2,y_train

In [7]:
df_test=pd.read_csv(test_csv)

In [8]:
xt1,xt2,yt=dataset_test(df_test)

In [9]:
xs1,xs2,ys=dataset_train(df_train)

In [85]:

def dist1(xy):
    x, y = xy
    sum_abs = K.sum(K.abs(x - y), axis=1, keepdims=True)
#     return sum_abs
    return K.maximum(sum_abs, K.epsilon())
def dist2(xy):
    x,y=xy
    sum_square=K.sum(K.square(x-y),axis=1,keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def dist3(xy):
    x,y=xy
    return K.sqrt(K.mean(K.square(x-y),axis=1,keepdims=True))
def dist4(xy):
    x,y=xy
    ss=K.sum(K.square(x-y),axis=1,keepdims=True)
    return K.sqrt(ss)/x.shape[1]


input1=keras.layers.Input(shape=(150,150,1))
# x=keras.layers.Flatten()(input1)
x=keras.layers.Conv2D(64,(10,10),activation='relu')(input1)
x=keras.layers.MaxPooling2D(2,2)(x)
# x=BatchNormalization()(x)
x=keras.layers.Dropout(0.5)(x)
# x=keras.layers.Conv2D(128,(7,7),activation='relu')(x)
# x=keras.layers.MaxPooling2D(2,2)(x)
x=keras.layers.Conv2D(100,(7,7),activation='relu')(x)
# x=BatchNormalization()(x)
x=keras.layers.MaxPooling2D(2,2)(x)
x=keras.layers.Dropout(0.5)(x)
# x=keras.layers.Conv2D(258,(4,4),activation='relu')(x)
# x=keras.layers.MaxPooling2D(2,2)(x)
x=keras.layers.Flatten()(x)
# x=keras.layers.Dense(4096,activation='relu')(x)
x=keras.layers.Dense(500,activation='relu')(x)
# x=BatchNormalization()(x)
dense=keras.models.Model(inputs=input1,outputs=x)


img1=keras.layers.Input(shape=(150,150,1))
img2=keras.layers.Input(shape=(150,150,1))
dense1=dense(img1)
dense2=dense(img2)
fc=keras.layers.Lambda(dist3)([dense1,dense2])
# fc=keras.layers.Dense(100,activation='relu')(fc)
fc=keras.layers.Dense(1,activation='sigmoid')(fc)

m=keras.models.Model(inputs=[img1,img2],outputs=fc)

m.compile(loss = "binary_crossentropy", optimizer="adam", metrics=["accuracy"])
m.summary()

    

Model: "functional_127"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_98 (InputLayer)           [(None, 150, 150, 1) 0                                            
__________________________________________________________________________________________________
input_99 (InputLayer)           [(None, 150, 150, 1) 0                                            
__________________________________________________________________________________________________
functional_125 (Functional)     (None, 500)          51520664    input_98[0][0]                   
                                                                 input_99[0][0]                   
__________________________________________________________________________________________________
lambda_31 (Lambda)              (None, 1)            0           functional_125[0][0]

In [86]:
m.fit([xs1,xs2],ys,epochs=10)

Epoch 1/10
63/63 [==============================] - 5s 77ms/step - loss: 0.6838 - accuracy: 0.5380
Epoch 2/10
63/63 [==============================] - 5s 80ms/step - loss: 0.5373 - accuracy: 0.5745
Epoch 3/10
63/63 [==============================] - 5s 78ms/step - loss: 0.4549 - accuracy: 0.6945
Epoch 4/10
63/63 [==============================] - 5s 77ms/step - loss: 0.3989 - accuracy: 0.8025
Epoch 5/10
63/63 [==============================] - 5s 77ms/step - loss: 0.3539 - accuracy: 0.8770
Epoch 6/10
63/63 [==============================] - 5s 76ms/step - loss: 0.3268 - accuracy: 0.9250
Epoch 7/10
63/63 [==============================] - 5s 77ms/step - loss: 0.3014 - accuracy: 0.9660
Epoch 8/10
63/63 [==============================] - 5s 78ms/step - loss: 0.2802 - accuracy: 0.9805
Epoch 9/10
63/63 [==============================] - 5s 78ms/step - loss: 0.2649 - accuracy: 0.9905
Epoch 10/10
63/63 [==============================] - 5s 77ms/step - loss: 0.2506 - accuracy: 0.9910


In [87]:
m.evaluate([xt1,xt2],yt)

63/63 [==============================] - 1s 18ms/step - loss: 0.3375 - accuracy: 0.9060


[0.3374583125114441, 0.906000018119812]

In [ ]:
def check(stored,image):
    img2=np.array([image])
    for i in range(0,stored.shape[0]):
        img1=np.array([stored[i]])
        pred=m.predict([img1,img2])
        print(pred)
        if(pred<0.5):
            print("matched",i)
            return
    print("unmatched")
    

In [ ]:
database_signatures=xt1[:15]

In [ ]:
check(database_signatures,xt1[13])

In [ ]:
plt.imshow(xs1[223])

In [ ]:
plt.imshow(xt1[223])